In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
path = '/home/zeus/Code/Arm-Sim/dataset/' # change to your path. 

features = np.load(path + 'features.npy', allow_pickle=False)
label = np.load(path + 'labels.npy', allow_pickle=False)

tensor_x = torch.Tensor(features).unsqueeze(1)
tensor_y = torch.Tensor(label).unsqueeze(1)

data = TensorDataset(tensor_x, tensor_y)

In [10]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.fc1 = nn.Sequential(
            nn.Conv2d(1,128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc2 = nn.Sequential(
            nn.Conv2d(128,256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc3 = nn.Linear(25600, 4)

    def forward(self, t):
        t = self.fc1(t)
        t = self.fc2(t)
        # print(t.shape)
        t = t.view(t.size(0), -1)
        # print(t.shape)
        t = self.fc3(t)
        # print(t.shape)

        return t

In [11]:

trainset, testset = random_split(data, [450000, 30000])

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NN()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [13]:
epoch = 100

for idx in range(epoch):
    for i, (data, label) in enumerate(trainloader):

        data = data.to(device)
        label = label.to(device)

        # data = data.view(data.size(0), -1)
        out = model(data)
        loss = criterion(data, label)
        # print(loss.shape)

        optimizer.zero_grad()      
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epochs {idx + 1}/{epoch}  Loss: {loss.item():.4f}')

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [128, 1, 5, 5], but got 2-dimensional input of size [32, 1600] instead

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
